In [1]:
from IPython.display import display, HTML
import pandas as pd
import sqlite3
from sqlite3 import Error

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows

conn = create_connection('la_city_crime.db', delete_db=True)

In [3]:
#Read dataset

df = pd.read_csv("Arrest_data.csv")

df = df.dropna()
print(len(df))
print(df.columns)


C:\Users\hp\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (21,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


414390
Index(['Report ID', 'Report Type', 'Arrest Date', 'Time', 'Area ID',
       'Area Name', 'Reporting District', 'Age', 'Sex Code', 'Descent Code',
       'Charge Group Code', 'Charge Group Description', 'Arrest Type Code',
       'Charge', 'Charge Description', 'Disposition Description', 'Address',
       'Cross Street', 'LAT', 'LON', 'Location', 'Booking Date',
       'Booking Time', 'Booking Location', 'Booking Location Code'],
      dtype='object')


In [4]:
sql_charge_detals = "CREATE TABLE ChargeDetails(ChargeGroupCode INTEGER NOT NULL PRIMARY KEY, ChargeGroupDesc TEXT NOT NULL, Charge TEXT NOT NULL, ChargeDesc TEXT NOT NULL, ChargeCount INTEGER NOT NULL);"

create_table(conn, sql_charge_detals, drop_table_name='ChargeDetails')

rows = []
charge_group_code = []
charge_group_desc = []
charge_code = []
charge_desc = []
charge_count = {}
df_charge_data = df[['Charge Group Code', 'Charge Group Description', 'Charge', 'Charge Description']]
print(df_charge_data.head(), len(df_charge_data), df_charge_data.index)

count = 0
for i in df_charge_data.index:
    count+=1
    try:
        charge_count[int(df_charge_data['Charge Group Code'][i])] +=1
    except KeyError:
        charge_count[int(df_charge_data['Charge Group Code'][i])] = 1
    if(int(df_charge_data['Charge Group Code'][i]) not in charge_group_code):
        charge_group_code += [int(df_charge_data['Charge Group Code'][i])]
        charge_group_desc += [df_charge_data['Charge Group Description'][i]]
        charge_code += [df_charge_data['Charge'][i] + ":"]
        charge_desc += [df_charge_data['Charge Description'][i] + ":"]

    else:
        index = charge_group_code.index(int(df_charge_data['Charge Group Code'][i]))
        if(df_charge_data['Charge'][i] not in charge_code[index]):
            charge_code[index] += df_charge_data['Charge'][i] + ":"
            charge_desc[index] += df_charge_data['Charge Description'][i] + ":"

print(charge_count)
    
for i in range(0, len(charge_group_code)):
    rows += [(int(charge_group_code[i]), charge_group_desc[i], charge_code[i][0:-1], charge_desc[i][0:-1],charge_count[int(charge_group_code[i])],)]

print(charge_group_code)
    
#print(len(charge_group_code), len(list(set(charge_group_code))))
#print(rows)
sql = """INSERT INTO ChargeDetails VALUES (?,?,?,?,?);"""
cur = conn.cursor()
cur.executemany(sql, rows)

conn.commit()
print(count)

      Charge Group Code Charge Group Description       Charge  \
331                 5.0                 Burglary        459PC   
673                 6.0                  Larceny        666PC   
784                 5.0                 Burglary        459PC   
2256               10.0       Fraud/Embezzlement  10980(C)2WI   
3108                7.0            Vehicle Theft   10851(A)VC   

                             Charge Description  
331                                    BURGLARY  
673    PETTY THEFT W/SPECIFIED PRIOR CONVICTION  
784                                    BURGLARY  
2256  OBTAIN/ETC AID BY FALSE STMENT/ETC > $400  
3108           TAKE VEHICLE W/O OWNER'S CONSENT   414390 Int64Index([    331,     673,     784,    2256,    3108,    4675,    4693,
               4922,    5175,   44678,
            ...
            1322791, 1322792, 1322793, 1322795, 1322798, 1322805, 1322808,
            1322814, 1322817, 1322819],
           dtype='int64', length=414390)
{5: 7650, 6: 105

In [11]:

import plotly.graph_objects as go
charge_by_group_count_data = execute_sql_statement("SELECT ChargeGroupDesc, ChargeCount FROM ChargeDetails ORDER BY ChargeCount DESC", conn)

charges = [row[0] for row in charge_by_group_count_data]
counts = [row[1] for row in charge_by_group_count_data]

fig = go.Figure([go.Bar(x=charges, y=counts,text=counts,
            textposition='auto')])
fig.update_layout(title_text='Commonly Committed Crimes In LA City')
fig.show()

fig = go.Figure(data=[go.Pie(labels=charges, values=counts)])
fig.show()

In [21]:
sql_location_details = "CREATE TABLE LocationDetails(AreaID INTEGER NOT NULL PRIMARY KEY, AreaName TEXT NOT NULL, ReportingDistrict TEXT NOT NULL, Address TEXT NOT NULL, CrossStreet TEXT NOT NULL, Location TEXT NOT NULL);"

create_table(conn, sql_location_details, drop_table_name='LocationDetails')

rows = []
area_id = []
area_name = []
reporting_district = []
address = []
cross_street = []
location = []
df_area_data = df[['Area ID', 'Area Name', 'Reporting District', 'Address', 'Cross Street', 'Location']]

for i in df_area_data.index:
    #print(i)
    if(int(df_area_data['Area ID'][i]) not in area_id):
        area_id += [int(df_area_data['Area ID'][i])]
        area_name += [df_area_data['Area Name'][i]]
        reporting_district += [str(df_area_data['Reporting District'][i]) + ":"]
        address += [str(df_area_data['Address'][i].replace(" ",'')) + ":"]
        cross_street += [df_area_data['Cross Street'][i].replace(" ",'') + ":"]
        location += [df_area_data['Location'][i].replace(" ",'') + ":"]
        

    else:
        index = area_id.index(int(df_area_data['Area ID'][i]))
        if(str(df_area_data['Reporting District'][i]) not in reporting_district[index]):
            reporting_district[index] += str(df_area_data['Reporting District'][i]) + ":"
        if(df_area_data['Address'][i].replace(" ",'') not in address[index]):
            address[index] += str(df_area_data['Address'][i].replace(" ",'')) + ":"
            cross_street[index] += df_area_data['Cross Street'][i].replace(" ",'') + ":"
            location[index] += df_area_data['Location'][i].replace(" ",'') + ":"

for i in range(0, len(area_id)):
    rows += [(int(area_id[i]), area_name[i], reporting_district[i][0:-1], address[i][0:-1], cross_street[i][0:-1], location[i][0:-1],)]
    
sql = """INSERT INTO LocationDetails VALUES (?,?,?,?,?,?);"""
cur = conn.cursor()
cur.executemany(sql, rows)

conn.commit()

In [22]:
df_area_data.head()

,Area ID,Area Name,Reporting District,Address,Cross Street,Location
331,12,77th Street,1267,MANCHESTER ST,FIGUEROA ST,POINT (-118.2827 33.9601)
673,12,77th Street,1255,75TH ST,BUDLONG AV,POINT (-118.2962 33.9719)
784,12,77th Street,1263,MANCHESTER,HARVARD,POINT (-118.3068 33.96)
2256,12,77th Street,1248,FLORENCE,MENLO,POINT (-118.2892 33.9746)
3108,12,77th Street,1215,49TH,FIGUEROA ST,POINT (-118.298 33.9921)


In [43]:
sql_arrest = '''create table ArrestDetails(ReportID INTEGER NOT NULL PRIMARY KEY, 
                                        ReportType TEXT NOT NULL, 
                                        ArrestDate TEXT NOT NULL,
                                        ArrestTime INTEGER NOT NULL, 
                                        ArrestTypeCode TEXT NOT NULL,
                                        Age INTEGER NOT NULL,
                                        SexCode TEXT NOT NULL,
                                        DescentCode TEXT NOT NULL,
                                        AreaID INTEGER NOT NULL,
                                        ChargeGroupCode INTEGER NOT NULL,
                                        FOREIGN KEY (AreaID) REFERENCES LocationDetails(AreaID),
                                        FOREIGN KEY (ChargeGroupCode) REFERENCES ChargeDetails(ChargeGroupCode)
                                        );
'''
create_table(conn, sql_arrest, drop_table_name = 'ArrestDetails')

rows = []
report_id = []
report_type = []
arrest_date = []
arrest_time = []
arrest_type_code = []
age = []
sex_code = []
descent_code = []
area_id = []
charge_gcode = []

df_arrest_data = df[['Report ID', 'Report Type', 'Arrest Date', 'Time', 'Arrest Type Code', 'Age', 'Sex Code', 'Descent Code', 'Area ID', 'Charge Group Code']]

for i in df_arrest_data.index[0:100000]:
    if(int(df_arrest_data['Report ID'][i]) not in report_id):
        report_id += [int(df_arrest_data['Report ID'][i])]
        report_type += [df_arrest_data['Report Type'][i]]
        arrest_date += [df_arrest_data['Arrest Date'][i]]
        arrest_time += [int(df_arrest_data['Time'][i])]
        arrest_type_code += [df_arrest_data['Arrest Type Code'][i]]
        age += [int(df_arrest_data['Age'][i])]
        sex_code += [df_arrest_data['Sex Code'][i]]
        descent_code += [df_arrest_data['Descent Code'][i]]
        area_id += [int(df_arrest_data['Area ID'][i])]
        charge_gcode += [int(df_arrest_data['Charge Group Code'][i])]


for i in range(0, len(report_id)):
    rows += [(int(report_id[i]), report_type[i], arrest_date[i], int(arrest_time[i]), arrest_type_code[i], int(age[i]), sex_code[i], descent_code[i], int(area_id[i]), int(charge_gcode[i]))]
    
sql = """INSERT INTO ArrestDetails VALUES (?,?,?,?,?,?,?,?,?,?);"""
cur = conn.cursor()
cur.executemany(sql, rows)

conn.commit()

In [28]:
df_arrest_data.head()

,Report ID,Report Type,Arrest Date,Time,Arrest Type Code,Age,Sex Code,Descent Code,Area ID,Charge Group Code
331,2219945,BOOKING,02/05/2010,2200.0,F,30,M,B,12,5.0
673,2298261,BOOKING,04/14/2010,2010.0,F,40,M,B,12,6.0
784,2317530,BOOKING,05/01/2010,2115.0,F,32,F,H,12,5.0
2256,4050074,BOOKING,08/02/2014,1235.0,F,26,F,B,12,10.0
3108,4442940,BOOKING,09/13/2015,800.0,F,31,M,B,12,7.0
